In [1]:
%pip install tensorflow librosa kaggle

Next, we'll set up the Kaggle API to download the dataset.

In [4]:
import os
from google.colab import files

# Upload your kaggle.json file
print("Please upload your kaggle.json file:")
files.upload()

# Create a directory for the Kaggle API key
!mkdir -p ~/.kaggle
# Move the uploaded file to the correct directory
!mv kaggle.json ~/.kaggle/
# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json
print("Kaggle API key has been set up.")

Please upload your kaggle.json file:


Saving kaggle.json to kaggle.json
Kaggle API key has been set up.


In [15]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import librosa
import numpy as np
import os

# Load the saved model
@st.cache_resource
def load_my_model():
    return tf.keras.models.load_model("spoken_digit_model.h5")

model = load_my_model()

# Load the padding value
try:
    with open("max_pad_len.txt", "r") as f:
        max_pad_len = int(f.read())
except FileNotFoundError:
    st.error("max_pad_len.txt not found. Please run the data preprocessing steps in the notebook first.")
    st.stop()

# App title
st.title("Spoken Digit Classifier")

# File uploader
uploaded_file = st.file_uploader("Upload a .wav audio file", type=["wav"])

if uploaded_file is not None:
    # To read file as bytes:
    audio_bytes = uploaded_file.getvalue()

    # Save the uploaded file temporarily to process with librosa
    temp_audio_path = "temp_audio.wav"
    with open(temp_audio_path, "wb") as f:
        f.write(audio_bytes)

    try:
        # Load audio file
        y, sr = librosa.load(temp_audio_path)

        # Extract MFCCs (40 coefficients)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

        # Pad the MFCCs
        if mfccs.shape[1] < max_pad_len:
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        elif mfccs.shape[1] > max_pad_len:
            # Trim if longer than max_pad_len (this shouldn't happen with the dataset, but good practice)
            mfccs = mfccs[:, :max_pad_len]


        # Reshape for the model
        mfccs = mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)

        # Make prediction
        prediction = model.predict(mfccs)
        predicted_digit = np.argmax(prediction)

        # Display the result
        st.write(f"Predicted Digit: {predicted_digit}")

    except Exception as e:
        st.error(f"Error processing audio file: {e}")

    finally:
        # Clean up the temporary file
        if os.path.exists(temp_audio_path):
            os.remove(temp_audio_path)

Writing app.py


In [16]:
%%writefile requirements.txt
streamlit
tensorflow
librosa
numpy

Writing requirements.txt


In [14]:
# Save the model
model.save("spoken_digit_model.h5")

print("Model saved as spoken_digit_model.h5")

Model saved as spoken_digit_model.h5


In [13]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 14s 168ms/step - accuracy: 0.1816 - loss: 6.2645 - val_accuracy: 0.6517 - val_loss: 1.3084
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 164ms/step - accuracy: 0.6447 - loss: 1.0384 - val_accuracy: 0.8733 - val_loss: 0.4410
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 164ms/step - accuracy: 0.8611 - loss: 0.4424 - val_accuracy: 0.9000 - val_loss: 0.3200
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 156ms/step - accuracy: 0.9088 - loss: 0.2706 - val_accuracy: 0.9350 - val_loss: 0.2435
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 164ms/step - accuracy: 0.9312 - loss: 0.1978 - val_accuracy: 0.9617 - val_loss: 0.1747
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 166ms/step - accuracy: 0.9553 - loss: 0.1330 - val_accuracy: 0.9550 - val_loss: 0.1997
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 20s 158ms/step - accuracy: 0.9533 - loss: 0.1328 - val_accuracy: 0.9650 - val_loss: 0.1766
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 21s 164ms/step - accuracy: 0.9594 - loss: 0.1237 - val_accu

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Get input shape from the training data
input_shape = X_train.shape[1:]

# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax') # 10 units for digits 0-9
])

# Print the model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 38, 97, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 19, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 19, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 46, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 23, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 23, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 11776)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,507,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,527,562 (5.83 MB)

 Trainable params: 1,527,562 (5.83 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

# Convert lists to NumPy arrays
X = np.array(padded_mfccs)
y = np.array(labels)

# Encode labels to one-hot categorical format
y_encoded = to_categorical(y)

# Reshape X for CNN (add a channel dimension)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (2400, 40, 99, 1)
Shape of X_test: (600, 40, 99, 1)
Shape of y_train: (2400, 10)
Shape of y_test: (600, 10)


In [10]:
# Pad MFCCs to max_pad_len
padded_mfccs = []
for mfccs in mfccs_list:
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    padded_mfccs.append(mfccs)

print(f"Padded MFCCs shape of first sample: {padded_mfccs[0].shape}")

# Save max_pad_len to a text file
with open("max_pad_len.txt", "w") as f:
    f.write(str(max_pad_len))

print(f"max_pad_len ({max_pad_len}) saved to max_pad_len.txt")

Padded MFCCs shape of first sample: (40, 99)
max_pad_len (99) saved to max_pad_len.txt


In [9]:
import librosa
import numpy as np
import os

# Directory containing the audio files
audio_dir = "/kaggle/input/free-spoken-digit-dataset-fsdd/recordings"

# Lists to store MFCCs and labels
mfccs_list = []
labels = []
max_pad_len = 0

# Loop through each file in the audio directory
for filename in os.listdir(audio_dir):
    if filename.endswith(".wav"):
        # Extract label from filename
        label = filename.split("_")[0]
        labels.append(label)

        # Load audio file
        filepath = os.path.join(audio_dir, filename)
        y, sr = librosa.load(filepath)

        # Extract MFCCs (40 coefficients)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

        # Update max_pad_len
        if mfccs.shape[1] > max_pad_len:
            max_pad_len = mfccs.shape[1]

        # Store MFCCs
        mfccs_list.append(mfccs)

print(f"Maximum padding length (max_pad_len): {max_pad_len}")
print(f"Number of audio files processed: {len(mfccs_list)}")

Maximum padding length (max_pad_len): 99
Number of audio files processed: 3000


In [7]:
import os

download_path = "/root/.cache/kagglehub/datasets/joserzapata/free-spoken-digit-dataset-fsdd/versions/1"
print(os.listdir(download_path))

['__init__.py', '.gitignore', 'acquire_data', 'utils', 'metadata.py', 'recordings', 'README.md', 'pip_requirements.txt']


In [8]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("joserzapata/free-spoken-digit-dataset-fsdd")

print(f"Dataset downloaded to {path}")

# The audio files are in the 'recordings' subdirectory
audio_dir = os.path.join(path, "recordings")
print(f"Audio files are located in: {audio_dir}")

Using Colab cache for faster access to the 'free-spoken-digit-dataset-fsdd' dataset.
Dataset downloaded to /kaggle/input/free-spoken-digit-dataset-fsdd
Audio files are located in: /kaggle/input/free-spoken-digit-dataset-fsdd/recordings


# Task
Update the Streamlit app (`app.py`) to include a live audio recording option alongside the existing file upload. The app should also display a visualization of the live audio input. Troubleshoot the "error processing audio" issue with uploaded files. Update `requirements.txt` with any new dependencies.

## Troubleshoot audio processing error

### Subtask:
Investigate why the uploaded audio files are causing an error in the Streamlit app. This might involve adding more detailed error logging to the app or testing the audio processing logic in the notebook.


**Reasoning**:
Add detailed error handling to the audio processing part of the Streamlit app to identify the cause of the error with uploaded files.



In [34]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import librosa
import numpy as np
import os
import traceback
# Removed: from streamlit_webrtc import webrtc_streamer, AudioProcessorBase, WebRtcMode
# Removed: import av
# Removed: import simplejson as json
import matplotlib.pyplot as plt
import seaborn as sns

# Load the saved model
@st.cache_resource
def load_my_model():
    return tf.keras.models.load_model("spoken_digit_model.h5")

model = load_my_model()

# Load the padding value
try:
    with open("max_pad_len.txt", "r") as f:
        max_pad_len = int(f.read())
except FileNotFoundError:
    st.error("max_pad_len.txt not found. Please run the data preprocessing steps in the notebook first.")
    st.stop()

# App title
st.title("Spoken Digit Classifier")

# Removed: Option to choose input method
# Removed: input_method = st.radio("Choose input method:", ("Upload Audio File", "Record Live Audio"))

# Directly use File uploader
uploaded_file = st.file_uploader("Upload a .wav audio file", type=["wav"])

if uploaded_file is not None:
    # To read file as bytes:
    audio_bytes = uploaded_file.getvalue()

    # Save the uploaded file temporarily to process with librosa
    temp_audio_path = "temp_audio.wav"
    with open(temp_audio_path, "wb") as f:
        f.write(audio_bytes)

    try:
        # Load audio file
        st.write("Loading audio file...")
        y, sr = librosa.load(temp_audio_path)
        st.write(f"Audio loaded successfully with sample rate: {sr}")
        st.write(f"Audio duration: {len(y)/sr:.2f} seconds")

        # Display audio player
        st.audio(audio_bytes, format='audio/wav')

        # Display audio waveform visualization
        st.subheader("Audio Waveform")
        fig, ax = plt.subplots()
        librosa.display.waveshow(y=y, sr=sr, ax=ax)
        st.pyplot(fig)
        plt.close(fig) # Close the figure to prevent it from displaying again


        # Extract MFCCs (40 coefficients)
        st.write("Extracting MFCCs...")
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        st.write(f"MFCCs shape: {mfccs.shape}")


        # Pad the MFCCs
        st.write(f"Padding MFCCs to length {max_pad_len}...")
        if mfccs.shape[1] < max_pad_len:
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
            st.write(f"Padded MFCCs shape: {mfccs.shape}")

        elif mfccs.shape[1] > max_pad_len:
            # Trim if longer than max_pad_len (this shouldn't happen with the dataset, but good practice)
            mfccs = mfccs[:, :max_pad_len]
            st.write(f"Trimmed MFCCs shape: {mfccs.shape}")


        # Reshape for the model
        mfccs = mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1)
        st.write(f"Reshaped MFCCs shape for model: {mfccs.shape}")

        # Make prediction
        st.write("Making prediction...")
        prediction = model.predict(mfccs)
        predicted_digit = np.argmax(prediction)

        # Display the result
        st.success(f"Predicted Digit: {predicted_digit}")

    except Exception as e:
        st.error(f"Error processing audio file: {e}")
        st.error(traceback.format_exc()) # Display detailed traceback

    finally:
        # Clean up the temporary file
        if os.path.exists(temp_audio_path):
            os.remove(temp_audio_path)

# Removed the entire elif block for "Record Live Audio"

Overwriting app.py


**Reasoning**:
Test the audio processing logic used in the Streamlit app within the notebook environment using a sample audio file to see if the error is reproducible outside the app context.



In [18]:
import librosa
import numpy as np
import os

# Define the path to a sample audio file from the dataset
# Assuming the dataset is downloaded to /kaggle/input/free-spoken-digit-dataset-fsdd
sample_audio_path = "/kaggle/input/free-spoken-digit-dataset-fsdd/recordings/0_jackson_0.wav"

# Load the max_pad_len from the saved file
try:
    with open("max_pad_len.txt", "r") as f:
        max_pad_len = int(f.read())
    print(f"Loaded max_pad_len: {max_pad_len}")
except FileNotFoundError:
    print("Error: max_pad_len.txt not found. Please run the data preprocessing steps.")


try:
    # Load audio file
    print(f"Loading audio file: {sample_audio_path}")
    y, sr = librosa.load(sample_audio_path)
    print(f"Audio loaded successfully with sample rate: {sr}")
    print(f"Audio duration: {len(y)/sr:.2f} seconds")

    # Extract MFCCs (40 coefficients)
    print("Extracting MFCCs...")
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    print(f"MFCCs shape: {mfccs.shape}")

    # Pad the MFCCs
    print(f"Padding MFCCs to length {max_pad_len}...")
    if mfccs.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs_padded = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        print(f"Padded MFCCs shape: {mfccs_padded.shape}")
    elif mfccs.shape[1] > max_pad_len:
        # Trim if longer than max_pad_len
        mfccs_padded = mfccs[:, :max_pad_len]
        print(f"Trimmed MFCCs shape: {mfccs_padded.shape}")
    else:
        mfccs_padded = mfccs
        print(f"MFCCs shape matches max_pad_len: {mfccs_padded.shape}")


    # Reshape for the model (optional for this test, but good to match app logic)
    mfccs_reshaped = mfccs_padded.reshape(1, mfccs_padded.shape[0], mfccs_padded.shape[1], 1)
    print(f"Reshaped MFCCs shape: {mfccs_reshaped.shape}")

    print("Audio processing steps completed successfully in the notebook.")

except Exception as e:
    print(f"Error processing audio file in notebook: {e}")
    import traceback
    print(traceback.format_exc())


Loaded max_pad_len: 99
Loading audio file: /kaggle/input/free-spoken-digit-dataset-fsdd/recordings/0_jackson_0.wav
Audio loaded successfully with sample rate: 22050
Audio duration: 0.64 seconds
Extracting MFCCs...
MFCCs shape: (40, 28)
Padding MFCCs to length 99...
Padded MFCCs shape: (40, 99)
Reshaped MFCCs shape: (1, 40, 99, 1)
Audio processing steps completed successfully in the notebook.


**Reasoning**:
The previous step showed that the audio processing logic works correctly in the notebook environment. This suggests the issue might be related to how Streamlit handles file uploads or the environment where the app is running. Since detailed error logging has been added to `app.py`, the next step is to run the Streamlit app and test the file upload functionality to observe the specific error messages generated by the enhanced error handling.



In [19]:
!streamlit run app.py & npx localtunnel --port 8501

/bin/bash: line 1: streamlit: command not found
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://sharp-ends-write.loca.lt
y
6789
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:6123 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙

**Reasoning**:
The Streamlit app failed to start because the `streamlit` command was not found. This indicates that Streamlit is not installed in the environment. The previous `pip install` command in cell `0083b053` did not include `streamlit`.



In [20]:
%pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.0 MB/s eta 0:00:00


**Reasoning**:
Streamlit has been installed. Now, re-run the command to start the Streamlit app and expose it via localtunnel to test the file upload functionality and observe the error messages with the added logging.



In [21]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼

⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://fair-papers-relax.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.129.142:8501

  Stopping...
^C


In [22]:
%pip install streamlit-webrtc pydub simplejson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: pyopenssl
    Found existing installation: pyOpenSSL 24.2.1
    Uninstalling pyOpenSSL-24.2.1:
      Successfully uninstalled pyOpenSSL-24.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [23]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.129.142:8501

your url is: https://lovely-banks-dream.loca.lt
  Stopping...
^C


In [31]:
from pyngrok import ngrok
ngrok.set_auth_token("31aLpXkl28KjC0U9TaqIvhOTfY6_5sRLVk3j2kp2uBuozWrcj")

!streamlit run app.py & ngrok http 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.129.142:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/lib/python3.12/subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 2053, in _wait
    (pid, sts) = self._try_wait(0)
  Stopping...
                 ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 2011, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pyngrok/ngrok.py", line 689, in main
    run(sys.argv[

In [35]:
%%writefile requirements.txt
streamlit
tensorflow
librosa
numpy
# Removed: streamlit-webrtc
pydub
simplejson
matplotlib
seaborn

Overwriting requirements.txt


In [36]:
%cat requirements.txt

streamlit
tensorflow
librosa
numpy
# Removed: streamlit-webrtc
pydub
simplejson
matplotlib
seaborn


In [28]:
!streamlit run app.py & ngrok http 8501


ERROR:  authentication failed: Usage of ngrok requires a verified account and authtoken.
ERROR:  
ERROR:  Sign up for an account: https://dashboard.ngrok.com/signup
ERROR:  Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_4018
ERROR:  https://ngrok.com/docs/errors/err_ngrok_4018
ERROR:  

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.129.142:8501

  Stopping...


In [27]:
%pip install pyngrok